# Deep Learning and Neural Networks
Deep Learning is a subset of machine learning algorithms. It is based on neural networks and is capable of learning complex patterns within the data. It's advantage over simple machine learning methods lies in the fact that it can be built by stacking a number of layers of neurons to discover heirarchical patterns of featurese within the data. Thus, it should be able to extract more complex features not otherwise detected by simpler machine learning algorithms

<img src="https://developer.ibm.com/developer/default/articles/an-introduction-to-deep-learning/images/AI-ML-DL.png" width=240 height=240 />

Image source: [IBM Developer](https://developer.ibm.com/articles/an-introduction-to-deep-learning/)

## Neural Networks
The name "Neural Networks" comes from the structure of neurons in our brains, where multiple neurons connect to each other in layers to perform their respective functions. Although the similarity is vague, the name still catches on. To underestand this concept, let's start with explaining what is a neuron.

#### The Neuron
The basic building block of neural network is the neuron (sometimes called perceptron). It can be thought of as simple functions that take inputs and convert them to outputs. This conversion function is known as the activation function. In each neuron, the following steps take place:

* Take the from each variable in the input layer and multiply it by a factor (weight)
* Sum the product of inputs and weights and add a bias term (b)
* Apply a function (activation function) to the final sum

For the sake of understanding how a neuron works, let's assume we have a data for predicting the metastasis of a primary hepatic tumor. We collected four variables to be used for our task. These are the alpha fetoprotein (AFP), number of risk genes, alanine aminotransferase (ALT) and albumin level. The outcome for each patient is determined by an oncologist and is given in the column "Met.". The figure below shows how to use the input variables (coded $X_1$ to $X_4$) to compute the predicted output using one neuron. Each variable is multiplied by a particular weight ($W_1$ to $W_4$), and the product of these multiplications are summed. A bias term ($b$) is added at the end. Then, an activation function is used to convert the result of activation function to outpus (here, I used sigmoid function, but any other activation function should work).

<img src="https://raw.githubusercontent.com/abdulrahman1123/analysis_examples/main/dl_neuron.png" width=800 height=256 />

#### Activation Functions
There is a number of activation functions that are commonly used. These include the sigmoid function and rectified linear unit (ReLU) function. Other activation functions include the the sigmoid function, tanh function, leaky ReLU function, step function and others. For further information, please refer to [THIS WEBSITE](https://towardsdatascience.com/activation-functions-neural-networks-1cbd9f8d91d6).

Sigmoid function is an S-shaped function that gives results between 0 and 1, and thus it is most appropriate when the outcome is binary. The ReLU function is a function that gives 0 for $x < 0$, and returns x when $x > 0$


<img src="https://raw.githubusercontent.com/abdulrahman1123/analysis_examples/main/dl_activations.png" width=400 height=140 />

#### Building Neural Networks
The most basic form of neural networks consists of three layers: the input layer, a hidden layer and an output layer. In the hidden layer, neurons are arranged so that they receive input from the input layer and apply their activation functions on the input. The results are then passed to the output layer to give the final output.

<img src="https://raw.githubusercontent.com/abdulrahman1123/analysis_examples/main/dl_NN.png" width=800 height=266 />

#### Finding the Best Parameters
The final goal is to find the best parameters that make sure that the output of your network ($P(y)$ or $\hat{y}$) is as close as possible to the actual outcome ($y$). So, we first need a method to compute how far off we are in our prediction, and then we try to minimize this difference. This function is known as the loss function (also called cost function), and there are many loss functions out there. The simplest of which is the regression loss function, and is given by $L(\hat{y}, y) = y-\hat{y}$. Mean squared error (MSE) is another loss function that takes the square root of difference and finds the average of that. It is better in that it works regardless of the sign of difference (because of the squaring) and because it takes the error of all cases into account (by averaging). It is given by:

$$MSE(y,\hat{y}) = \frac{\sum_{i=0}^{n}(y-\hat{y})^2} {n}$$

Loss functions are used to validate the model and modify the weights and bias accordingly. The way it does that is through gradient descent, which basically updates the weights and bias so that the final result is closer to the actual outcome. This process is repeated a number of times until the loss function decreases and the best weights and bias are obtained.

<img src="https://raw.githubusercontent.com/abdulrahman1123/analysis_examples/main/dl_loss.png" width=400 height=210 />

---
### About the Dataset Used
This is a heart disease database, collected from pateints referred to the angiography clinic in Cleveland, US between 1981 and 1984. The original dataset consists of 76 variables. But most studies that used this dataset used a subset of 14, which we will use today. For futher information, please refer to the [following paper](https://pubmed.ncbi.nlm.nih.gov/2756873/)


---
# Neural Networks using Python
### Import data
Import data from ucimlrepo and take a look at the variables

In [3]:
from ucimlrepo import fetch_ucirepo

# fetch dataset 
heart_disease = fetch_ucirepo(id=45)

# data (as pandas dataframes)
X = heart_disease.data.features
y = heart_disease.data.targets

heart_disease.variables

,name,role,type,demographic,description,units,missing_values
0,age,Feature,Integer,Age,None,years,no
1,sex,Feature,Categorical,Sex,None,None,no
2,cp,Feature,Categorical,None,None,None,no
3,trestbps,Feature,Integer,None,resting blood pressure (on admission to the ho...,mm Hg,no
4,chol,Feature,Integer,None,serum cholestoral,mg/dl,no
5,fbs,Feature,Categorical,None,fasting blood sugar > 120 mg/dl,None,no
6,restecg,Feature,Categorical,None,None,None,no
7,thalach,Feature,Integer,None,maximum heart rate achieved,None,no
8,exang,Feature,Categorical,None,exercise induced angina,None,no
9,oldpeak,Feature,Integer,None,ST depression induced by exercise relative to ...,None,no


---
### Further reading:
[IBM Developer website](https://developer.ibm.com/articles/an-introduction-to-deep-learning/)

[Activation Functions in Neural Networks](https://towardsdatascience.com/activation-functions-neural-networks-1cbd9f8d91d6)

[Introduction to Deep Learning](https://www.geeksforgeeks.org/introduction-deep-learning/)